In [ ]:
import requests
import json
import itertools
import os
import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import random
from tqdm import tqdm
import time

# Read tokens from a text file
tokens_file = "./env/tokens.txt"
with open(tokens_file, "r") as file:
    tokens = file.read().splitlines()

# Create an iterator to cycle through the tokens
token_iterator = itertools.cycle(tokens)
current_token = next(token_iterator)

In [ ]:
# List of User-Agents for randomization
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
]

# Define headers to authenticate using the first token
headers = {
    "Authorization": f"Bearer {current_token}",
    "User-Agent": random.choice(user_agents),
}

# Setup GraphQL endpoint and client
graphql_url = "https://api.github.com/graphql"
transport = RequestsHTTPTransport(url=graphql_url, headers=headers, use_json=True)
client = Client(transport=transport, fetch_schema_from_transport=True)

In [ ]:
def log_activity(activity: str):
    log = f"{datetime.datetime.now()}: {activity}\n"
    # print(log)
    with open("non-spam-prs-output.log", "a") as log_file:
        log_file.write(log)

In [ ]:
# Test all tokens to verify their validity
def test_all_tokens():
    test_query = gql(
        """
        {
          viewer {
            login
          }
        }
        """
    )
    for i, token in enumerate(tokens):
        headers = {
            "Authorization": f"Bearer {token}",
            "User-Agent": random.choice(user_agents),
        }
        transport = RequestsHTTPTransport(
            url=graphql_url, headers=headers, use_json=True
        )
        client = Client(transport=transport, fetch_schema_from_transport=True)

        try:
            response = client.execute(test_query)
            log_activity(
                f"Token {i+1}/{len(tokens)} is valid. Logged in as: {response['viewer']['login']}"
            )
        except Exception as e:
            log_activity(f"Token {i+1}/{len(tokens)} failed with error: {e}")


# Run the token validation
test_all_tokens()

In [ ]:
# Define the GraphQL query
query_template = gql(
    """
    query searchIssues($keyword: String!, $afterCursor: String, $first: Int) {
      search(query: $keyword, type: ISSUE, first: $first, after: $afterCursor) {
        issueCount
        edges {
          cursor
          node {
            ... on PullRequest {
              id
              number
              title
              url
              comments(first: 100) {
                totalCount # it still gives the total count regardless of the first parameter
                edges {
                  node {   
                    author { ... on User { login } }
                    editor { ... on User { login } }
                    body
                    createdAt
                    lastEditedAt
                  }
                }
              }
              state
              closed
              merged
              createdAt
              updatedAt
              mergeCommit {
                oid
              }
              timeline(last: 100) {
                edges {
                  node {
                    __typename
                    ... on LabeledEvent {
                      actor { ... on User { login } }
                      label { ... on Label { name }}
                      createdAt
                    }
                    ... on ClosedEvent { 
                      actor { ... on User { login } }
                      createdAt
                    }
                  }
                }
              }
              commits {
                totalCount
              }
              changedFiles
              headRefName
              baseRefName
              repository {
                id
                nameWithOwner
                stargazerCount
                description
                codeOfConduct {
                  body
                  id
                  name
                  url
                }
                homepageUrl
                assignableUsers(first: 100) {
                  edges {
                    node {
                      login
                      url
                      bio
                      company
                    }
                  }
                  totalCount
                }
                mentionableUsers(first: 100) {
                  edges {
                    node {
                      login
                      url
                      bio
                      company
                    }
                  }
                  totalCount
                }
                forkCount
                watchers {
                  totalCount
                }
                isFork
                languages(first: 20) {
                  edges {
                    node {
                      name
                    }
                  }
                }
              }
              author {
                ... on User {
                  login
                  url
                  createdAt
                  repositories {
                    totalCount
                  }
                  followers {
                    totalCount
                  }
                  following {
                    totalCount
                  }
                  repositoryDiscussions {
                    totalCount
                  }
                  repositoryDiscussionComments {
                    totalCount
                  }
                  organizations (first: 20){
                    edges {
                      node {
                        name
                        login
                        url
                        membersWithRole {
                          totalCount
                        }
                      }
                    }
                  }
                }
              }
              labels(first: 10) {
                edges {
                  node {
                    name
                  }
                }
              }
              body
            }
          }
        }
        pageInfo {
          endCursor
          hasNextPage
        }
      }
    }
    """
)

In [ ]:
# def get_contributor_count(repo_owner, repo_name):
#     global current_token
#     max_retries = 3
#     retries = 0
#     while retries < max_retries:
#         try:
#             # Randomize User-Agent for each query
#             headers["User-Agent"] = random.choice(user_agents)
#             headers["Authorization"] = f"Bearer {current_token}"
#             url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contributors?per_page=1&anon=true"
#             response = requests.get(url, headers=headers)
#             if response.status_code == 200:
#                 return int(response.headers.get("Link", "").split(",")[-1].split("&page=")[-1].split(">")[0]) if "Link" in response.headers else len(response.json())
#             elif response.status_code == 403:
#                 log_activity(f"Rate limit exceeded, switching token... (Attempt {retries + 1}/{max_retries})")
#                 current_token = next(token_iterator)
#                 retries += 1
#             else:
#                 response.raise_for_status()
#         except Exception as e:
#             log_activity(f"Error: {e}, retrying... (Attempt {retries + 1}/{max_retries})")
#             retries += 1
#     raise Exception("Max retries reached. Unable to complete the request.")

In [ ]:
transport.headers = headers
# Check rate limit before executing the main query
rate_limit_query = gql(
    """
    query {
      viewer {
        login
      }
      rateLimit {
        limit
        remaining
        used
        resetAt
      }
    }
    """
)
rate_limit_response = client.execute(rate_limit_query)
log_activity(f"Rate limit: {rate_limit_response['rateLimit']}")

In [ ]:
rate_limit_query = gql(
    """
    query {
      viewer {
        login
      }
      rateLimit {
        limit
        remaining
        used
        resetAt
      }
    }
    """
)


def execute_query(keyword, first=100, after_cursor=None):
    global current_token
    log_activity(
        f"Executing query with keyword: {keyword}, first: {first}, afterCursor: {after_cursor}"
    )
    while True:
        try:
            # Randomize User-Agent for each query
            headers["User-Agent"] = random.choice(user_agents)
            transport.headers = headers
            # Check rate limit before executing the main query
            rate_limit_response = client.execute(rate_limit_query)
            remaining = rate_limit_response["rateLimit"]["remaining"]
            if remaining < 100:
                log_activity(
                    f"Rate limit remaining ({remaining}) is below threshold. Switching token..."
                )
                # Set up to track whether we have cycled through all tokens
                all_tokens_checked = False
                initial_token = current_token

                while not all_tokens_checked:
                    # Switch to the next token
                    current_token = next(token_iterator)
                    headers["Authorization"] = f"Bearer {current_token}"
                    transport.headers = headers

                    # Check the rate limit of the new token
                    rate_limit_response = client.execute(rate_limit_query)
                    remaining = rate_limit_response["rateLimit"]["remaining"]

                    if remaining >= 100:
                        log_activity(
                            f"Switched to a new token with sufficient rate limit ({remaining} remaining)."
                        )
                        break

                    # Check if we have cycled through all tokens
                    if current_token == initial_token:
                        log_activity("All tokens are below threshold. Waiting for 1 hour...")
                        time.sleep(3600)
                        all_tokens_checked = True

                continue

            return client.execute(
                query_template,
                variable_values={
                    "keyword": keyword,
                    "first": first,
                    "afterCursor": after_cursor,
                },
            )
        except Exception as e:
            if "API rate limit" in str(e):
                log_activity(
                    f"Rate limit reached: {e}, switching token... (Attempt with first {first})"
                )
                current_token = next(token_iterator)
                headers["Authorization"] = f"Bearer {current_token}"
            else:
                if first > 1:
                    first = max(1, first // 2)
                    log_activity(
                    f"Error: {e}, reducing number of results and retrying... (Attempt with first {first})"
                    )
                else:
                    log_activity(f"Query failed completely after retries: {e}")
                    break
    # Track consecutive failures
    abandon_threshold = 3
    if 'failure_count' not in globals():
        global failure_count
        failure_count = 0
    failure_count += 1
    if failure_count >= abandon_threshold:
        log_activity(f"Query failed {abandon_threshold} times in a row. Abandoning operation.")
        failure_count = 0  # Reset failure count after abandoning
        return {"abandon": True}
    log_activity("Max retries reached. Sleeping for 30 minutes and switching token...")
    time.sleep(3)
    # time.sleep(1800)
    current_token = next(token_iterator)
    headers["Authorization"] = f"Bearer {current_token}"
    transport.headers = headers
    return execute_query(keyword, first, after_cursor)

In [56]:

def get_create_close_max_range_dates(pull_requests):
    """
    Get the maximum range of creation and closing dates for pull requests grouped by repository.

    This function processes a list of pull requests and calculates the earliest creation date
    and the latest closing date for each repository. It returns a list of tuples containing
    the repository name, the earliest creation date, and the latest closing date.

    Args:
        pull_requests (list): A list of dictionaries, where each dictionary represents a pull request.
            Each dictionary should have the keys "repository_name_with_owner", "created_at", and "closed_at".

    Returns:
        list: A list of tuples, where each tuple contains:
            - str: The repository name.
            - datetime.datetime: The earliest creation date of pull requests in the repository.
            - datetime.datetime: The latest closing date of pull requests in the repository.

    Example:
        pull_requests = [
            {"repository_name_with_owner": "repo1", "created_at": "2023-01-01T12:00:00Z", "closed_at": "2023-01-02T12:00:00Z"},
            {"repository_name_with_owner": "repo1", "created_at": "2023-01-03T12:00:00Z", "closed_at": "2023-01-04T12:00:00Z"},
            {"repository_name_with_owner": "repo2", "created_at": "2023-01-01T12:00:00Z", "closed_at": "2023-01-05T12:00:00Z"},
        ]
        result = get_create_close_max_range_dates(pull_requests)
        # result: [('repo1', datetime.datetime(2023, 1, 1, 12, 0), datetime.datetime(2023, 1, 4, 12, 0)),
        #          ('repo2', datetime.datetime(2023, 1, 1, 12, 0), datetime.datetime(2023, 1, 5, 12, 0))]
    """
    if not pull_requests:  # Handle empty or None input
        return []

    date_ranges = {}
    for pr in pull_requests:
        repo = pr.get("repository_name_with_owner")
        created_at = pr.get("created_at")
        closed_at = pr.get("closed_at")

        if not repo or not created_at or not closed_at:
            continue

        created_at = datetime.datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%SZ")
        closed_at = datetime.datetime.strptime(closed_at, "%Y-%m-%dT%H:%M:%SZ")

        if repo not in date_ranges:
            date_ranges[repo] = (created_at, closed_at)
        else:
            current_min, current_max = date_ranges[repo]
            date_ranges[repo] = (
                min(current_min, created_at),
                max(current_max, closed_at),
            )

    return [
        {
            "repository_name_with_owner": repo,
            "start_date": dates[0],
            "end_date": dates[1],
        }
        for repo, dates in date_ranges.items()
    ]

In [57]:
import pickle

if os.path.exists("non-spam-progress.pkl"):
    with open("non-spam-progress.pkl", "rb") as f:
        progress_data = pickle.load(f)
        df = progress_data["df"]
        start_index = progress_data["start_index"]
else:
    df = []
    start_index = 0

In [58]:
# df = []
# start_index = 0

In [59]:
start_date_param = "2023-12-17T17:02"
repos_to_skip = [
    "daiyi/blog",
    "thoemmi/thomasfreudenberg.com",
    "faithworkcamps/faithworkcamps.github.io",
]

In [ ]:
import datetime
import pickle


def execute_with_dynamic_date_range(
    execute_query,
    process_results,
    index,
    se_fm_repository_data,
    repo,
    days_offset=31,
    max_total_allowed_results=950,
    default_days_interval=60,
):
    """
    Executes a GraphQL query within dynamically adjusted date ranges to handle large datasets.

    :param keywords: List of keywords for search queries.
    :param execute_query: Function to execute the query.
    :param process_results: Function to process the query results.
    :param max_total_allowed_results: Max allowed results before reducing date range.
    :param default_days_interval: Initial days interval for date range.
    """
    repository_name_with_owner = repo.get("repository_name_with_owner")
    earliest_pr_create_date = repo.get("start_date")
    latest_pr_close_date = repo.get("end_date")

    if (
        not repository_name_with_owner
        or not earliest_pr_create_date
        or not latest_pr_close_date
    ):
        return

    start_date = earliest_pr_create_date - datetime.timedelta(days=days_offset)
    end_date = latest_pr_close_date + datetime.timedelta(days=days_offset)
    days_interval = default_days_interval

    while start_date < end_date:
        next_date_candidate = start_date + datetime.timedelta(days=days_interval)
        next_date = min(next_date_candidate, end_date)

        try:
            after_cursor = None
            while True:
                date_range = f"{start_date.strftime('%Y-%m-%dT%H:%M')}..{next_date.strftime('%Y-%m-%dT%H:%M')}"
                search_keyword = f"-label:spam repo:{repository_name_with_owner} is:pr is:public archived:false created:{date_range}"
                response = execute_query(
                    search_keyword, first=10, after_cursor=after_cursor
                )

                if "abandon" in response and response["abandon"]:
                    days_interval = default_days_interval  # Reset interval
                    break
                log_activity(f'response count: {response["search"]["issueCount"]}\n')

                if response["search"]["issueCount"] == 0:
                    days_interval = default_days_interval  # Reset interval
                    break

                # Adjust interval if issue count exceeds max allowed
                if response["search"]["issueCount"] > max_total_allowed_results:
                    reduced_interval = (
                        max(1, days_interval // 2)
                        if days_interval > 1
                        else max(0.00069, days_interval / 2)
                    )
                    log_activity(f"Reducing interval to {reduced_interval} days...")
                    days_interval = reduced_interval
                    next_date = start_date + datetime.timedelta(days=days_interval)
                    continue

                # Process results
                process_results(response)

                # Pagination
                page_info = response["search"]["pageInfo"]
                if page_info["hasNextPage"]:
                    after_cursor = page_info["endCursor"]
                else:
                    break
            with open("non-spam-progress.pkl", "wb") as f:
                pickle.dump({"df": se_fm_repository_data, "start_index": index + 1}, f)

            # Reset interval to default after a successful run
            days_interval = default_days_interval
        except Exception as e:
            log_activity(
                f"Error fetching data for '{search_keyword}' in range {date_range}: {e}"
            )
            # Save progress before terminating
            with open("non-spam-progress.pkl", "wb") as f:
                pickle.dump({"df": df, "start_index": index}, f)
            raise

        start_date = next_date  # Move to the next date interval

In [61]:
def get_most_spammed_pr_repositories(pull_requests, max=10):
    """
    Get the repositories with the most spam pull requests.

    This function processes a list of pull requests and calculates the number of spam pull requests
    for each repository. It returns a list of dictionaries containing the repository name and the
    count of spam pull requests, sorted in descending order by the count.

    Args:
        pull_requests (list): A list of dictionaries, where each dictionary represents a pull request.
            Each dictionary should have the keys "repository_name_with_owner" and "labeled_spam_by".
        max (int): The maximum number of repositories to return. Default is 10.

    Returns:
        list: A list of dictionaries, where each dictionary contains:
            - str: The repository name.
            - int: The count of spam pull requests in the repository.

    Example:
        pull_requests = [
            {"repository_name_with_owner": "repo1", "labeled_spam_by": "user1"},
            {"repository_name_with_owner": "repo1", "labeled_spam_by": "user2"},
            {"repository_name_with_owner": "repo2", "labeled_spam_by": "user1"},
        ]
        result = get_most_spammed_pr_repositories(pull_requests)
        # result: [{'repository_name_with_owner': 'repo1', 'spam_pr_count': 2},
        #          {'repository_name_with_owner': 'repo2', 'spam_pr_count': 1}]
    """
    if not pull_requests:  # Handle empty or None input
        return []

    spam_tracker = {}
    for pr in pull_requests:
        repo = pr.get("repository_name_with_owner")

        if not repo:
            continue

        if repo not in spam_tracker:
            spam_tracker[repo] = {"count": 0, "details": []}
        spam_tracker[repo]["count"] += 1
        spam_tracker[repo]["details"].append(pr)

    sorted_spam_counts = sorted(
        spam_tracker.items(), key=lambda x: x[1]["count"], reverse=1
    )
    return [
        {"prs": repo_data["details"], "spam_pr_count": repo_data["count"]}
        for _, repo_data in sorted_spam_counts[:max]
    ]

In [62]:
def result_processor(
    response,
):
    for edge in response["search"]["edges"]:
        pull_request = edge["node"]

        if not pull_request:
            continue
        timeline = pull_request["timeline"]["edges"]
        labeled_spam_event = next(
            filter(
                lambda x: x["node"]
                and x["node"]["__typename"] == "LabeledEvent"
                and x["node"]["label"]["name"]
                and (x["node"]["label"]["name"]).lower()
                in ["ai spam", "spam", "ai-spam"],
                timeline,
            ),
            None,
        )
        labeled_spam_event_node = (
            labeled_spam_event["node"] if labeled_spam_event else None
        )
        closed_event = next(
            filter(
                lambda x: x["node"] and x["node"]["__typename"] == "ClosedEvent",
                timeline,
            ),
            None,
        )
        closed_event_node = closed_event["node"] if closed_event else None
        author = pull_request["author"]
        comments = [comment["node"] for comment in pull_request["comments"]["edges"]]
        labeled_spam_at = (
            labeled_spam_event_node["createdAt"] if labeled_spam_event_node else None
        )

        comments_by_spam_labeler = []

        closed_by = (
            closed_event_node["actor"]["login"]
            if closed_event_node and closed_event_node["actor"]
            else None
        )
        end_date = closed_event_node["createdAt"] if closed_event_node else None
        comments_by_closer = [
            {
                **closer_comment,
                "commented_before_closing": (
                    closer_comment["createdAt"] < end_date if end_date else False
                ),
            }
            for closer_comment in comments
            if (
                (
                    closer_comment["author"]
                    and closer_comment["author"]["login"] == closed_by
                )
                or (not closer_comment["author"] and not closed_by)
            )
        ]

        author_organizations = (
            [
                organization["node"]
                for organization in author["organizations"]["edges"]
                if organization["node"]
            ]
            if author
            and author.get("organizations")
            and author["organizations"].get("edges")
            else []
        )

        timestamp_suffix = f"_as_at_{datetime.datetime.now().strftime('%Y-%m-%d')}"

        df.append(
            {
                "id": pull_request["id"],
                "title": pull_request["title"],
                "url": pull_request["url"],
                "state": pull_request["state"],
                "comments_count": pull_request["comments"]["totalCount"],
                "comments_by_spam_labeler_count": len(comments_by_spam_labeler),
                "comments_by_spam_labeler": comments_by_spam_labeler,
                "labeled_spam_by": (
                    labeled_spam_event_node["actor"]["login"]
                    if labeled_spam_event_node and labeled_spam_event_node["actor"]
                    else None
                ),
                "is_labeled_spam_by_bot": False,
                "labeled_spam_at": labeled_spam_at,
                "comments_by_closer_count": len(comments_by_closer),
                "comments_by_closer": comments_by_closer,
                "closed": pull_request["closed"],
                "is_closed_by_bot": closed_by is None and pull_request["closed"],
                "closed_by": closed_by,
                "closed_at": end_date,
                "merged": pull_request["merged"],
                "body": pull_request["body"],
                "created_at": pull_request["createdAt"],
                "updated_at": pull_request["updatedAt"],
                "repository": pull_request["repository"],
                "repository_name_with_owner": pull_request["repository"][
                    "nameWithOwner"
                ],
                "repository_stargazer_count": pull_request["repository"][
                    "stargazerCount"
                ],
                "repository_watcher_count": pull_request["repository"]["watchers"][
                    "totalCount"
                ],
                "repository_is_fork": pull_request["repository"]["isFork"],
                "repository_languages": [
                    language["node"]["name"]
                    for language in pull_request["repository"]["languages"]["edges"]
                ],
                "merge_commit": (
                    pull_request["mergeCommit"]["oid"]
                    if pull_request["mergeCommit"]
                    else None
                ),
                "labels": [
                    label["node"]["name"] for label in pull_request["labels"]["edges"]
                ],
                "commits_count": pull_request["commits"]["totalCount"],
                "changed_files_count": pull_request["changedFiles"],
                "author_name": (author["login"] if author else None),
                "author_url": (author["url"] if author else None),
                "author_account_created_at": (author["createdAt"] if author else None),
                f"author_repository_count{timestamp_suffix}": (
                    author["repositories"]["totalCount"]
                    if author and author["repositories"]
                    else None
                ),
                f"author_followers_count{timestamp_suffix}": (
                    author["followers"]["totalCount"]
                    if author and author["followers"]
                    else None
                ),
                f"author_following_count{timestamp_suffix}": (
                    author["following"]["totalCount"]
                    if author and author["following"]
                    else None
                ),
                f"author_repository_discussions_count{timestamp_suffix}": (
                    author["repositoryDiscussions"]["totalCount"]
                    if author and author["repositoryDiscussions"]
                    else None
                ),
                f"author_repository_discussion_comments_count{timestamp_suffix}": (
                    author["repositoryDiscussionComments"]["totalCount"]
                    if author and author["repositoryDiscussionComments"]
                    else None
                ),
                f"author_organizations{timestamp_suffix}": author_organizations,
            }
        )


with open("progress.pkl", "rb") as f:
    data = pickle.load(f)
    spam_prs = data["df"]


# template
index = start_index
se_fm_repository_data = df
most_spammed_repos = get_most_spammed_pr_repositories(spam_prs, 10)
# Extract top pull requests from the most spammed repositories
top_prs = []
for repo in most_spammed_repos:
    for pr in repo["prs"]:
        top_prs.append(pr)

repos = get_create_close_max_range_dates(top_prs)

has_used_start_date_param = False
for repo in repos:

    if repo.get("repository_name_with_owner") in repos_to_skip:
        continue
    if not has_used_start_date_param:
        repo["start_date"] = datetime.datetime.strptime(
            start_date_param, "%Y-%m-%dT%H:%M"
        )
        has_used_start_date_param = True
    execute_with_dynamic_date_range(
        execute_query=execute_query,
        process_results=result_processor,
        repo=repo,
        index=start_index,
        se_fm_repository_data=df,
    )

KeyboardInterrupt: 

In [ ]:
# with open("progress.pkl", "rb") as f:
#     data = pickle.load(f)
#     spam_prs = data["df"]


# # template
# index = start_index
# se_fm_repository_data = df
# most_spammed_repos = get_most_spammed_pr_repositories(spam_prs)
# # Extract top pull requests from the most spammed repositories
# top_prs = []
# for repo in most_spammed_repos:
#     for pr in repo["prs"]:
#         top_prs.append(pr)

# repos = get_create_close_max_range_dates(top_prs)

# DAYS_OFFSET = 31

# for repo in repos:
#     repository_name_with_owner = repo.get("repository_name_with_owner")
#     earliest_pr_create_date = repo.get("start_date")
#     latest_pr_close_date = repo.get("end_date")

#     if (
#         not repository_name_with_owner
#         or not earliest_pr_create_date
#         or not latest_pr_close_date
#     ):
#         continue
#     start_date = (earliest_pr_create_date - datetime.timedelta(days=DAYS_OFFSET)).strftime(
#         "%Y-%m-%d"
#     )
#     end_date = (latest_pr_close_date + datetime.timedelta(days=DAYS_OFFSET)).strftime("%Y-%m-%d")
#     search_keyword = f"-label:spam repo:{repository_name_with_owner} is:pr is:public archived:false created:{start_date}..{end_date}"

#     try:
#         after_cursor = None
#         while True:
#             response = execute_query(
#                 search_keyword, first=10, after_cursor=after_cursor
#             )

#             if response["search"]["issueCount"] == 0:
#                 break
#             # Extract pr
#             for edge in response["search"]["edges"]:
#                 pull_request = edge["node"]

#                 if not pull_request:
#                     continue
#                 timeline = pull_request["timeline"]["edges"]
#                 labeled_spam_event = next(
#                     filter(
#                         lambda x: x["node"]
#                         and x["node"]["__typename"] == "LabeledEvent"
#                         and x["node"]["label"]["name"]
#                         and (x["node"]["label"]["name"]).lower() == "spam",
#                         timeline,
#                     ),
#                     None,
#                 )
#                 labeled_spam_event_node = (
#                     labeled_spam_event["node"] if labeled_spam_event else None
#                 )
#                 closed_event = next(
#                     filter(
#                         lambda x: x["node"]
#                         and x["node"]["__typename"] == "ClosedEvent",
#                         timeline,
#                     ),
#                     None,
#                 )
#                 closed_event_node = closed_event["node"] if closed_event else None
#                 author = pull_request["author"]
#                 comments = [
#                     comment["node"] for comment in pull_request["comments"]["edges"]
#                 ]
#                 labeled_spam_at = (
#                     labeled_spam_event_node["createdAt"]
#                     if labeled_spam_event_node
#                     else None
#                 )

#                 labeled_spam_by = (
#                     labeled_spam_event_node["actor"]["login"]
#                     if labeled_spam_event_node and labeled_spam_event_node["actor"]
#                     else None
#                 )
#                 comments_by_spam_labeler = []

#                 closed_by = (
#                     closed_event_node["actor"]["login"]
#                     if closed_event_node and closed_event_node["actor"]
#                     else None
#                 )
#                 end_date = closed_event_node["createdAt"] if closed_event_node else None
#                 comments_by_closer = [
#                     {
#                         **closer_comment,
#                         "commented_before_closing": (
#                             closer_comment["createdAt"] < end_date
#                             if end_date
#                             else False
#                         ),
#                     }
#                     for closer_comment in comments
#                     if (
#                         (
#                             closer_comment["author"]
#                             and closer_comment["author"]["login"] == closed_by
#                         )
#                         or (not closer_comment["author"] and not closed_by)
#                     )
#                 ]

#                 author_organizations = (
#                     [
#                         organization["node"]
#                         for organization in author["organizations"]["edges"]
#                         if organization["node"]
#                     ]
#                     if author
#                     and author.get("organizations")
#                     and author["organizations"].get("edges")
#                     else []
#                 )

#                 timestamp_suffix = (
#                     f"_as_at_{datetime.datetime.now().strftime('%Y-%m-%d')}"
#                 )

#                 df.append(
#                     {
#                         "id": pull_request["id"],
#                         "title": pull_request["title"],
#                         "url": pull_request["url"],
#                         "state": pull_request["state"],
#                         "comments_count": pull_request["comments"]["totalCount"],
#                         "comments_by_spam_labeler_count": len(comments_by_spam_labeler),
#                         "comments_by_spam_labeler": comments_by_spam_labeler,
#                         "labeled_spam_by": (
#                             labeled_spam_event_node["actor"]["login"]
#                             if labeled_spam_event_node
#                             and labeled_spam_event_node["actor"]
#                             else None
#                         ),
#                         "is_labeled_spam_by_bot": False,
#                         "labeled_spam_at": labeled_spam_at,
#                         "comments_by_closer_count": len(comments_by_closer),
#                         "comments_by_closer": comments_by_closer,
#                         "closed": pull_request["closed"],
#                         "is_closed_by_bot": closed_by is None
#                         and pull_request["closed"],
#                         "closed_by": closed_by,
#                         "closed_at": end_date,
#                         "merged": pull_request["merged"],
#                         "body": pull_request["body"],
#                         "created_at": pull_request["createdAt"],
#                         "updated_at": pull_request["updatedAt"],
#                         "repository": pull_request["repository"],
#                         "repository_name_with_owner": pull_request["repository"][
#                             "nameWithOwner"
#                         ],
#                         "repository_stargazer_count": pull_request["repository"][
#                             "stargazerCount"
#                         ],
#                         "repository_watcher_count": pull_request["repository"][
#                             "watchers"
#                         ]["totalCount"],
#                         "repository_is_fork": pull_request["repository"]["isFork"],
#                         "repository_languages": [
#                             language["node"]["name"]
#                             for language in pull_request["repository"]["languages"][
#                                 "edges"
#                             ]
#                         ],
#                         "merge_commit": (
#                             pull_request["mergeCommit"]["oid"]
#                             if pull_request["mergeCommit"]
#                             else None
#                         ),
#                         "labels": [
#                             label["node"]["name"]
#                             for label in pull_request["labels"]["edges"]
#                         ],
#                         "commits_count": pull_request["commits"]["totalCount"],
#                         "changed_files_count": pull_request["changedFiles"],
#                         "author_name": (author["login"] if author else None),
#                         "author_url": (author["url"] if author else None),
#                         "author_account_created_at": (
#                             author["createdAt"] if author else None
#                         ),
#                         f"author_repository_count{timestamp_suffix}": (
#                             author["repositories"]["totalCount"]
#                             if author and author["repositories"]
#                             else None
#                         ),
#                         f"author_followers_count{timestamp_suffix}": (
#                             author["followers"]["totalCount"]
#                             if author and author["followers"]
#                             else None
#                         ),
#                         f"author_following_count{timestamp_suffix}": (
#                             author["following"]["totalCount"]
#                             if author and author["following"]
#                             else None
#                         ),
#                         f"author_repository_discussions_count{timestamp_suffix}": (
#                             author["repositoryDiscussions"]["totalCount"]
#                             if author and author["repositoryDiscussions"]
#                             else None
#                         ),
#                         f"author_repository_discussion_comments_count{timestamp_suffix}": (
#                             author["repositoryDiscussionComments"]["totalCount"]
#                             if author and author["repositoryDiscussionComments"]
#                             else None
#                         ),
#                         f"author_organizations{timestamp_suffix}": author_organizations,
#                     }
#                 )

#             # Pagination
#             page_info = response["search"]["pageInfo"]
#             if page_info["hasNextPage"]:
#                 after_cursor = page_info["endCursor"]
#             else:
#                 break
#         with open("non-spam-progress.pkl", "wb") as f:
#             pickle.dump({"df": se_fm_repository_data, "start_index": index + 1}, f)

#     except Exception as e:
#         log_activity(f"Failed to retrieve data for search_keyword '{search_keyword}': {e}")
#         # Save progress before terminating
#         with open("non-spam-progress.pkl", "wb") as f:
#             pickle.dump({"df": df, "start_index": index}, f)
#         raise

In [ ]:
import pandas as pd
import pickle

# Load the .pkl file
with open("non-spam-progress.pkl", "rb") as file:
    data = pickle.load(file)

# Check if it's a list of dictionaries
if isinstance(data["df"], list) and all(isinstance(d, dict) for d in data["df"]):
    # Convert to DataFrame
    df = pd.DataFrame(data["df"])
    
    # Remove duplicates by 'id'
    df = df.drop_duplicates(subset="id", keep="first")

    # Convert back to a list of dictionaries
    cleaned_data = {**data, "df": df.to_dict(orient="records")}

# Check if it's already a DataFrame
elif isinstance(data["df"], pd.DataFrame):
    # Remove duplicates by 'id'
    cleaned_df = data["df"].drop_duplicates(subset="id", keep="first")
    cleaned_data = {**data, "df": cleaned_df}

# Save the cleaned data
with open("non-spam-progress.pkl", "wb") as file:
    pickle.dump(cleaned_data, file)

log_activity("Duplicates removed and data saved successfully.")

In [ ]:
import pandas as pd


def save_pkl_content_as_csv_and_json(filepath):
    """
    Display the content of a pickle file and save it as CSV and JSON files.

    This function reads a pickle file from the given filepath, prints its content,
    and saves the data contained in the "df" key to both a CSV file and a JSON file.
    The CSV file is saved with the name "spam_data_without_org_join_date.csv" and the JSON file is saved
    with the name "spam_data_without_org_join_date.json".

    Args:
        filepath (str): The path to the pickle file to be read.

    Raises:
        Exception: If there is an error reading the pickle file or writing the CSV/JSON files.
    """
    try:
        with open(filepath, "rb") as f:
            data = pickle.load(f)

        log_activity(f"Content of {filepath}:\n")
        filename = "non-spam_data_without_org_join_date"
        pd.DataFrame(data["df"]).to_csv(f"{filename}.csv", index=True)
        log_activity(f"Data written to {filename}.csv successfully.")
        try:
            with open(f"{filename}.json", "w") as f:
                json.dump(data["df"], f, indent=4)
            log_activity(f"Data written to {filename}.json successfully.")
        except Exception as e:
            log_activity(f"An error occurred: {e}")
    except Exception as e:
        log_activity(f"An unexpected error occurred: {e}")


filepath = "non-spam-progress.pkl"
save_pkl_content_as_csv_and_json(filepath)

In [ ]:
# generate metadata
df = []
start_index = 0



def generate_metadata(filepath):
    """ """
    try:
        with open(filepath, "rb") as f:
            data = pickle.load(f)

        log_activity(f"Content of {filepath}:\n")
        filename = "non-spam_data.meta"
        pull_requests = data["df"]
        unique_repository = {}
        unique_pr_author = {}
        unique_pr_spam_labeler = {}
        unique_pr_closer = {}
        merged_pr_count = 0
        closed_pr_count = 0
        for pull_request in pull_requests:
            def update_unique_value_dict(info_dict, key, value):
                if not value:
                    log_activity(f"Warning: Pull request missing '{key}' {pull_request}")
                    return False
                if value not in info_dict:
                    info_dict[value] = value
                return True

            # Update repository count
            update_unique_value_dict(unique_repository, "repository_name_with_owner", pull_request["repository_name_with_owner"])

            # Update author count
            update_unique_value_dict(unique_pr_author, "author_name", pull_request["author_name"])

            # Update spam labeler count
            update_unique_value_dict(unique_pr_spam_labeler, "labeled_spam_by", pull_request["labeled_spam_by"])

            # Update closer count
            update_unique_value_dict(unique_pr_closer, "closed_by", pull_request["closed_by"])

            merged_pr_count += 1 if pull_request["merged"] else 0
            closed_pr_count += 1 if pull_request["closed"] is not None else 0


        total_prs= len(pull_requests)
        unique_repository_count= len(unique_repository)
        unique_pr_author_count= len(unique_pr_author)
        unique_pr_spam_labeler_count= len(unique_pr_spam_labeler)
        unique_pr_closer_count= len(unique_pr_closer)
        
        df.append(
            {
            "total_prs": total_prs,
            "unique_repository_count": unique_repository_count,
            "unique_pr_author_count": unique_pr_author_count,
            "unique_pr_author_ratio": round(unique_pr_author_count / total_prs, 3),
            "unique_pr_spam_labeler_count": unique_pr_spam_labeler_count,
            "unique_pr_spam_labeler_ratio": round(unique_pr_spam_labeler_count / total_prs, 3),
            "unique_pr_closer_count": unique_pr_closer_count,
            "unique_pr_closer_ratio": round(unique_pr_closer_count / total_prs, 3),
            "merged_pr_count": merged_pr_count,
            "merged_pr_ratio": round(merged_pr_count / total_prs, 3),
            "closed_pr_count": closed_pr_count,
            "closed_pr_ratio": round(closed_pr_count / total_prs, 3),
            }
        )

        pd.DataFrame(df).to_csv(f"{filename}.csv", index=True)
        log_activity(f"Data written to {filename}.csv successfully.")
        try:
            with open(f"{filename}.json", "w") as f:
                json.dump(df, f, indent=4)
            log_activity(f"Data written to {filename}.json successfully.")
        except Exception as e:
            log_activity(f"An error occurred: {e}")
    except Exception as e:
        log_activity(f"An unexpected error occurred: {e}")


filepath = "non-spam-progress.pkl"
generate_metadata(filepath)